In [1]:
%matplotlib inline

import itertools
from collections import Counter

import numpy as np
import pandas as pd

In [2]:
TEST_RUN = False

In [3]:
train_df = pd.read_csv(
    "./local_train_splitByUser", sep='\t',
    names=['label', 'uid', 'mid', 'cat', 'hist_mids', 'hist_cats'])

valid_df = pd.read_csv(
    "./local_test_splitByUser", sep='\t',
    names=['label', 'uid', 'mid', 'cat', 'hist_mids', 'hist_cats'])

In [4]:
item_info_df = pd.read_csv("./item-info", sep='\t', names=['mid', 'cat'])

In [5]:
item_info_df.head()

,mid,cat
0,0001048791,Books
1,0001048775,Books
2,0001048236,Books
3,0000401048,Books
4,0001019880,Books


In [6]:
reviews_info_df = pd.read_csv("./reviews-info", sep='\t', names=['c1', 'mid', 'c3', 'c4'])

In [7]:
reviews_info_df.head()

,c1,mid,c3,c4
0,A10000012B7CGYKOMPQ4L,000100039X,5.0,1355616000
1,A2S166WSCFIFP5,000100039X,5.0,1071100800
2,A1BM81XB4QHOA3,000100039X,5.0,1390003200
3,A1MOSTXNIO5MPJ,000100039X,5.0,1317081600
4,A2XQ5LZHTD4AFT,000100039X,5.0,1033948800


In [8]:
reviews_info_df = reviews_info_df[['mid']].merge(item_info_df, on='mid', how='inner').drop_duplicates()

In [9]:
reviews_info_df.head()

,mid,cat
0,000100039X,Books
206,0001055178,Books
224,0001473123,Books
240,0001473727,Books
247,0001473905,Books


In [10]:
mid_cat_map = reviews_info_df.set_index('mid').to_dict()['cat']

In [11]:
if TEST_RUN:
    train_df = train_df.sample(1000)
    valid_df = valid_df.sample(1000)

In [12]:
# slow implement
def prepare_neg(df):
    records = df['hist_mids'].apply(lambda x: x.split(""))
    candidates = list(mid_cat_map.keys())
    max_len = len(candidates)

    def neg_sampling(filters, length):    
        mids = []
        cats = []
        for i in range(length):
            while(1):
                c = candidates[np.random.randint(0, max_len)]
                if c not in filters:
                    mids.append(c)
                    cats.append(mid_cat_map[c])
                    filters.add(c)
                    break
        return mids, cats
    
    total_neg_mids = []
    total_neg_cats = []
    for record in records:
        neg_mids, neg_cats = neg_sampling(set(record), len(record))
        total_neg_mids.append("".join(neg_mids))
        total_neg_cats.append("".join(neg_cats))
    
    return total_neg_mids, total_neg_cats

In [13]:
total_neg_mids, total_neg_cats = prepare_neg(train_df)
train_df['neg_hist_mids'] = total_neg_mids
train_df['neg_hist_cats'] = total_neg_cats

In [14]:
total_neg_mids, total_neg_cats = prepare_neg(valid_df)
valid_df['neg_hist_mids'] = total_neg_mids
valid_df['neg_hist_cats'] = total_neg_cats

In [15]:
train_df.to_csv('local_train.csv', sep='\t', index=False)
valid_df.to_csv('local_test.csv', sep='\t', index=False)